<a href="https://colab.research.google.com/github/wtyn/AI-Practice-Tensorflow-Notes/blob/master/%E4%B8%89%E4%B8%AA%E7%B1%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 运行此单元格即可装载您的 Google 云端硬盘。
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 数据

In [0]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import random


image_size = 197
class_num = 3
is_gpu = True

In [0]:

def parse_exmp(serial_exmp):
    # filename_queue = tf.train.string_input_producer(file_list)  # 建立一个队列，其中的参数为tfrecords文件的路径
    # reader = tf.TFRecordReader()  # 实例化读操作，建立读取器
    # _, serialized_example = reader.read(filename_queue)  # 返回文件名和文件

    keys_to_features = {
        'image': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64)
    }

    feats = tf.parse_single_example(serial_exmp, keys_to_features)

    # image = tf.decode_raw(feats['image'], tf.uint8)
    # label = feats['label']

    image = tf.decode_raw(feats['image'], tf.uint8)
    
    label = tf.cast(feats['label'], tf.int64)
    
    return image, label


def _preprocess_image(image, label):
    
    _image = preprocess_input(image)
    
    return _image, label
    


def get_dataset(file_list, shuffle_num=0, batch_size=32):
    
    
    dataset = tf.data.TFRecordDataset(file_list)

    # Maps the parser on every filepath in the array. You can set the number of parallel loaders here
    dataset = dataset.map(parse_exmp)
    
    # 使用lambda函数处理(归一化)
#     dataset = dataset.map(lambda image, label : (preprocess_input(image), label))
#     dataset = dataset.map(lambda image, label: preprocess_image(image, label))
    
#     dataset = dataset.map(_preprocess_image)
    
    
    
    # This dataset will go on forever

    dataset = dataset.repeat()

    # Set the number of datapoints you want to load and shuffle
    if shuffle_num > 0:
        dataset = dataset.shuffle(shuffle_num)

    # Set the batchsize

    dataset = dataset.batch(batch_size)

    # Create an iterator

    iterator = dataset.make_one_shot_iterator()

    # Create your tf representation of the iterator
    
    image, label = iterator.get_next()
    
    
    # Bring your picture back in shape
    
    image = tf.reshape(image, [-1, image_size, image_size, class_num])
    image = tf.cast(image, tf.float32)
    
    image =tf.keras.applications.resnet50. preprocess_input(image)
    
    # Create a one hot array for your labels
    
    label = tf.one_hot(label, class_num)
    
    return image, label






In [5]:


# 数据集
data_file = '/content/drive/My Drive/dataset/三分类/cats_dogs_panda_197_197.tfrecords'

x, y= get_dataset([data_file], batch_size=1, shuffle_num=0)


x_ = []
y_ = []

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    for i in range(3000):
        image, label = sess.run((x, y))
        
        
        x_.append(image)
        y_.append(label)

        
x_ = np.array(x_).reshape((-1, image_size, image_size, 3))


y_ = np.array(y_).reshape((-1, class_num))

print(x_.shape)
print(y_.shape)


# 打乱数据集
random.seed(88)
index = [i for i in range(len(x_))]
random.shuffle(index)

x_ = x_[index]
y_ = y_[index]


(x_train, x_test, y_train, y_test) = train_test_split(x_,
                                                  y_, test_size=0.25, random_state=88)

print('------------')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_train[0])





W0629 05:37:48.239406 139712764553088 deprecation.py:323] From <ipython-input-4-18c37120dd4e>:62: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


(3000, 197, 197, 3)
(3000, 3)
------------
(2250, 197, 197, 3)
(2250, 3)
(750, 197, 197, 3)
(750, 3)
[[[ -58.939003   -47.779      -12.68     ]
  [ -57.939003   -39.779      -15.68     ]
  [ -38.939003   -12.778999    -5.6800003]
  ...
  [ -94.939     -103.779      -91.68     ]
  [ -59.939003   -42.779      -30.68     ]
  [ -61.939003   -66.779      -56.68     ]]

 [[ -26.939003   -11.778999     5.3199997]
  [ -36.939003   -22.779       -2.6800003]
  [ -56.939003   -43.779      -14.68     ]
  ...
  [ -84.939      -68.779      -60.68     ]
  [ -90.939      -84.779      -78.68     ]
  [ -62.939003   -46.779      -48.68     ]]

 [[ -33.939003   -13.778999     5.3199997]
  [ -32.939003   -11.778999     4.3199997]
  [ -38.939003   -25.779        3.3199997]
  ...
  [ -78.939      -89.779      -72.68     ]
  [ -67.939      -61.779      -53.68     ]
  [-101.939      -81.779      -85.68     ]]

 ...

 [[ -64.939      -11.778999   -67.68     ]
  [ -62.939003    -0.7789993  -59.68     ]
  [ -36.9

# 模型

In [0]:
class Trainsport_resnet50:
    
    @staticmethod
    def build():
        
        # base_model = ResNet50(weights='imagenet', include_top=False, pooling=None,
        #                       input_shape=(img_weight, img_height, color), classes=nb_classes)
        base_model_weights_path = '/content/drive/My Drive/model/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
        base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, 
                              input_shape=(image_size, image_size, 3), 
                              pooling='avg',
                              classes=class_num)

        for layer in base_model.layers:
            layer.trainable = False

        x = base_model.output

#         x = Flatten()(x)

#         x = GlobalAveragePooling2D()(x)
#         x = tf.keras.layers.Dropout(0.5)(x)
        
#         x = tf.keras.layers.Dense(320, activation='relu',
#                  kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01))(x)
        
#         x = tf.keras.layers.Dropout(0.5)(x)
        
        
        predictions = tf.keras.layers.Dense(class_num, activation='softmax', 
                            kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01))(x)

        inputs = base_model.input
        outputs = predictions

        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        
#         model.summary()
        
        return model

# 训练

In [0]:

val_loss = 1000.


# 定义回调
class LossHistory(tf.keras.callbacks.Callback):
    
        
    def on_train_begin(self, logs):
        self.losses = []
        self.acces = []
        self.val_losses = []
        self.val_acces = []
    
        
    def on_epoch_end(self, batch, logs):
        print('-----')
        print(logs)
        self.losses.append(logs.get('loss'))
        self.acces.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        self.val_acces.append(logs.get('val_acc'))
        
#         print('\nloss:',self.losses[-1])
        global val_loss
        if logs.get('val_loss') < val_loss:
            val_loss = logs.get('val_loss')
            self.model.save('cats_dogs_min_loss_model.h5')
            print(val_loss)
        



In [0]:

H = None

BATCH_SIZE = 64
EPOCHES = 500


if is_gpu:
    
    model = Trainsport_resnet50.build()
    
    adam = tf.keras.optimizers.Adam(1e-4)
    
    model.compile(loss="categorical_crossentropy",  
              optimizer=adam,
	          metrics=["accuracy"])


#     es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint('cats_dogs_best_model.h5', monitor='val_loss', save_best_only=True)
    

    
#     lossHistory = LossHistory()
#     lossHistory.set_model(model)
    
    aug = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=90, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest"
                           )
    
    
    
    H = model.fit(aug.flow(x_train, y_train), 
            epochs=EPOCHES, 
            validation_data=(x_test, y_test), 
            callbacks=[mc]
         )
    
#     H = model.fit(x_train, y_train, 
#             epochs=EPOCHES, 
#             validation_data=(x_test, y_test), 
#             callbacks=[mc]
#          )
    
    
    
   

#     H = model.fit(x_train, y_train,
#                    epochs=100, 
#                    steps_per_epoch=100, 
#                    validation_data=(x_test, y_test),
#                    validation_steps=4004//32,
#                    validation_freq=5
#                   )
 

# model.fit(train_x, train_y, epochs=20, steps_per_epoch=100, validation_data=(test_x, test_y), validation_freq=10)


    




94658560/94653016 [==============================] - 1s 0us/step


W0629 05:41:49.405825 139712764553088 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:94: calling TruncatedNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/500
71/71 [==============================] - 31s 440ms/step - loss: 0.8246 - acc: 0.7400 - val_loss: 0.3748 - val_acc: 0.9467
Epoch 2/500
71/71 [==============================] - 18s 256ms/step - loss: 0.5095 - acc: 0.8996 - val_loss: 0.1950 - val_acc: 0.9773
Epoch 3/500
71/71 [==============================] - 19s 265ms/step - loss: 0.3869 - acc: 0.9147 - val_loss: 0.1422 - val_acc: 0.9747
Epoch 4/500
71/71 [==============================] - 19s 268ms/step - loss: 0.3317 - acc: 0.9169 - val_loss: 0.1024 - val_acc: 0.9800
Epoch 5/500
71/71 [==============================] - 19s 263ms/step - loss: 0.2788 - acc: 0.9356 - val_loss: 0.0892 - val_acc: 0.9813
Epoch 6/500
71/71 [==============================] - 19s 265ms/step - loss: 0.2546 - acc: 0.9324 - val_loss: 0.0827 - val_acc: 0.9760
Epoch 7/500
71/71 [==============================] - 19s 263ms/step - loss: 0.2426 - acc: 0.9302 - val_loss: 0.0719 - val_acc: 0.9827
Epoch 8/500
71/71 [==============================] - 19s 262ms

# TPU训练

In [0]:
import os



# tpu训练
if not is_gpu:
    
    
    
    TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

    resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu=TPU_WORKER)

    tf.contrib.distribute.initialize_tpu_system(resolver)

    strategy = tf.contrib.distribute.TPUStrategy(resolver)


    with strategy.scope():
        model = Trainsport_resnet50.build()

        model.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(),
	          metrics=["accuracy"])


# model.fit(dataset_train, 
#           epochs=20, 
#           steps_per_epoch=100, 
#           validation_data=dataset_test, 
#           validation_steps=20, 
#           validation_freq=5
#          )
    

#     H = model.fit(x_train, y_train, 
#           epochs=50, 
#           steps_per_epoch=105, 
#           validation_data=(x_test, y_test), 
#           validation_steps=4004//32, 
#           validation_freq=5
#          )
    
    count = len(x_train)//32*32
    x_train = x_train[:count]
    y_train = y_train[:count]
    H = model.fit(x_train, y_train, 
          epochs=50, 
          steps_per_epoch=70
          
         )




# 评估

In [0]:
# 保存模型到本地
print("[INFO] 正在保存模型")
model.save('cats_dogs_classify_model.h5')

# f = open('label.bin', "wb")
# f.write(pickle.dumps(labl.bin))
# f.close()

In [0]:
import matplotlib.pyplot as plt

# 画图
def plot_training(history):
        acc = history.history['acc']
        val_acc = history.history['val_acc']
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        epochs = range(len(acc))
        plt.plot(epochs, acc, 'b-')
        plt.plot(epochs, val_acc, 'r')
        plt.title('Training and validation accuracy')
        plt.figure()
        
        plt.plot(epochs, loss, 'b-')
        plt.plot(epochs, val_loss, 'r-')
        plt.title('Training and validation loss')
        plt.show()
        

plot_training(H)


In [0]:


# 测试网络模型
def evaluation(model_, x_test, y_test):
    
#     x_test = preprocess_input(x_test)
    
    
    print("[INFO] 正在评估模型")

    predictions = model_.predict(x_test, batch_size=200)

    
    evaluation_score = classification_report(y_test.argmax(axis=1),
                                             predictions.argmax(axis=1), target_names=['cats', 'dogs', 'panda'])

    print(evaluation_score)

evaluation(model, x_test, y_test)

print('----->>>>>>')

# 最好模型预测
model_best = Trainsport_resnet50.build()
model_best.load_weights('cats_dogs_best_model.h5')
evaluation(model_best, x_test, y_test)



# 直接加载模型进行评估

In [0]:
model_ = tf.keras.models.load_model('cats_dogs_best_model.h5')

In [0]:
loss, score = model_.evaluate(x_test, y_test, verbose=0)
print('now val loss¡ is %f, score is %f'%(loss, score))